In [157]:
import numpy as np
import math
import sympy as sym
from scipy.integrate import ode
from scipy.io import loadmat
import scipy.io
import scipy
from scipy import stats
import pickle
import copy
import filter_functions
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt
from IPython.core.debugger import Tracer
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 4.0)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
np.set_printoptions(precision=15)
sym.init_printing()
from IPython.display import display

#MSIS: https://github.com/DeepHorizons/Python-NRLMSISE-00
#import time
from nrlmsise_00_header import *
from nrlmsise_00 import *
#SUBROUTINE GTD7D -- d[5] is the "effective total mass density
#for drag" and is the sum of the mass densities of all species
#in this model, INCLUDING anomalous oxygen.

#define constants
r_earth_const = 6378136.3 * 1e-3 #km
omega_const = 7.2921158553e-5 #rad/s, angular velocity of earth
J_2_const = .00108262617385222
J_3_const = -.00000253241051856772
mu_earth = 3.986004415e14 * 1e-9 #km^3/s^2


#Drag:
A_const = 0.9551567 * 1e-6 #km^2; cross-sectional area of satellite
m_const = 10 #kg; mass of satellite
C_D_const = 2.0
theta_dot_const = 7.2921158553e-5 #rad/sec

In [158]:


meas_type = 3

if meas_type == 1:
    meas_file = open('Data Files/meas_range_rangeRate.pkl', 'rb')
elif meas_type == 2:
    meas_file = open('Data Files/meas_az_el.pkl', 'rb')
elif meas_type == 3:
    meas_file = open('Data Files/meas_az_el_range_geoMagStorm_new.pkl', 'rb') #_10s_all_3stat.pkl
    
    
    
#Date of Simulation Details:
#June 24th, 2017 at 6am (this is the date & time at the beginning of the simulation/orbit)
year_init = 2017
month_init = 6
day_of_month_init = 24
day_of_year_init = 175
hour_init = 6
boulder_UT_offset = 6 #Boulder time + 6 hours = UT time
hour_init_UT = hour_init + boulder_UT_offset
    
    

#Canbera Station (DSS 34)
lat_dss34 = math.radians(-35.398333)
lon_dss34 = math.radians(148.981944)
alt_dss34 = 691.75 * 1e-3 #km

r_ecef_dss34 = filter_functions.topo2ecef(lat_dss34, lon_dss34, alt_dss34, r_earth_const)
#print(r_ecef_dss34)

#Madrid Station (DSS 65) -- correct position of Madrid Station
lat_dss65 = math.radians(40.427222)
lon_dss65 = math.radians(355.749444)
alt_dss65 = 834.539 * 1e-3 #km

r_ecef_dss65 = filter_functions.topo2ecef(lat_dss65, lon_dss65, alt_dss65, r_earth_const)
#print(r_ecef_dss65)

#Goldstone Station (DSS 13) 
lat_dss13 = math.radians(35.247164)
lon_dss13 = math.radians(200.205)
alt_dss13 = 1071.14904 * 1e-3 #km

r_ecef_dss13 = filter_functions.topo2ecef(lat_dss13, lon_dss13, alt_dss13, r_earth_const)
#print(r_ecef_dss13)

#Diego Garcia, British Indian Ocean Territory 7.41173°S 72.45222°E., Space Fence (Dedicated Sensor
lat_diego = math.radians(-7.41173)
lon_diego = math.radians(72.45222)
alt_diego = 0 * 1e-3 #km, "sea level"

r_ecef_diego = filter_functions.topo2ecef(lat_diego, lon_diego, alt_diego, r_earth_const)




# read python dict containing measurements
mydict2 = pickle.load(meas_file)
meas_file.close()
measurement_array = mydict2['measurement_array']
truth_xyz = mydict2['truth_pos_vel']
true_density_array = mydict2['true_density']*1e9
lat_lst_meas_array = mydict2['lat_lst_array']
print(np.shape(measurement_array))
print(np.shape(truth_xyz))

#convert to km
truth_xyz = truth_xyz * 1e-3
measurement_array[:, -1] = measurement_array[:, -1] * 1e-3

"""
measurement_array = measurement_array[4000:,:]
truth_xyz = truth_xyz[4000:, :]
true_density_array = true_density_array[4000:]
lat_lst_meas_array = lat_lst_meas_array[4000:, :]
"""


print(int(len(measurement_array)  * 5/12))
print(true_density_array[:50])

(5873, 5)
(5873, 6)
2447
[ 0.006220279271195  0.006320297154409  0.006422421647351
  0.006526071102665  0.006630641787903  0.006735531050316
  0.006840154382354  0.006943954660357  0.007046402762124
  0.007146989833883  0.00724521253494   0.007340553498266
  0.007432459889754  0.007520327720051  0.007603468367547
  0.007681126285407  0.007752462066215  0.007816567466764
  0.007872486487474  0.007919245972165  0.007955894005453
  0.007981543689478  0.00799541942126   0.007996902595564
  0.007985573726384  0.007961248286443  0.007924004059057
  0.007874198415012  0.007812474597546  0.007739756753372
  0.007657234031866  0.007566334551604  0.007468690387579
  0.007366094967373  0.00726045439812   0.007153734303929
  0.007047903762658  0.006944877911909  0.006846460764349
  0.006754289737506  0.006669783368964  0.006594093657076
  0.006528064445124  0.006472197258902  0.006426626016034
  0.006391102042202  0.006364990830735  0.006347281925015
  0.006336613129501  0.006331306687284]


In [159]:
#read in files necessary for ensemble & density est. portion


# read python dict containing densities
ensemble_file = open('Data Files/ensemble_density_grids_moreRealistic.pkl', 'rb')
mydict2 = pickle.load(ensemble_file)
ensemble_file.close()

#shape: ensembles (# of combos) by lat by lon
ensembles_of_density_grid = mydict2['ensemble_of_density_grids'] 
print('Shape of Vector of State/density Ensembles:', np.shape(ensembles_of_density_grid))
(num_of_ensembles, num_of_lat, num_of_lon) = np.shape(ensembles_of_density_grid)

#convert from kg/m**3 to kg/km**3 -> 1/(1e-3)**3 = 1/(1e-9) = 1e9
ensembles_of_density_grid = ensembles_of_density_grid * 1e9

latitude_grid = mydict2['latitudes'] 
longitude_grid = mydict2['longitudes'] 
lat_res = latitude_grid[1] - latitude_grid[0] #spacing between each latitude "tick"
lon_res = longitude_grid[1] - longitude_grid[0] #spacing between each longitude "tick"



#add noise w/ standard deviation = 1e-4 (used as initialization of density covariance, as well)

print(np.sum(ensembles_of_density_grid[:, 0, 0])/num_of_ensembles)
ensemble_noise = np.random.randn(num_of_ensembles, num_of_lat, num_of_lon) * 1e-4#1e-9 #1e-4****
ensembles_of_density_grid = ensembles_of_density_grid + ensemble_noise
print(np.sum(ensembles_of_density_grid[:, 0, 0])/num_of_ensembles) 

print()

Shape of Vector of State/density Ensembles: (450, 37, 73)
0.00351022831235
0.00350896749951



In [160]:

#Propogate reference trajectory and S.T.M.
def orbitpropogator_EnKF(t, X_vector, density):
    
    ensemble_member = X_vector

    #find X acceleration via the F(X) lambdified equation
    state_acc = X_dot_sol_fcn(ensemble_member[0], ensemble_member[1], ensemble_member[2], \
                                  ensemble_member[3], ensemble_member[4], ensemble_member[5], density)
        
    dx = state_acc.flatten()
    return dx


In [161]:

#EnKF specific functionality




def gen_ensemble(X_0, Bsqrt_cov, ensemble_size):
    
    X_0 = X_0.reshape(len(X_0), 1)
    
    ensemble = np.zeros((len(X_0), ensemble_size))
    
    for ii in range(ensemble_size):
        
        member = X_0 + np.dot(Bsqrt_cov, np.random.randn(len(X_0), 1))

        ensemble[:, ii] = member.reshape(len(X_0))
    
    return ensemble
    



In [162]:
#two body motion force
# **Setup force equations/acceleration/U

#Force equations with J_2
x, y, z, J_2, r_earth, mu, r, J_3 = sym.symbols('x y z J_2 r_earth mu r J_3')


two_body_J2_string = 'mu/r * ( 1 - J_2*(r_earth/r)**2 * (3/2 * (z/r)**2 - 1/2) )' #potential
two_body_J2 = sym.sympify(two_body_J2_string)
two_body_J2 = two_body_J2.subs([(r, sym.sqrt(x**2+y**2+z**2))])
two_body_J2_acc_x = two_body_J2.diff(x)
two_body_J2_acc_y = two_body_J2.diff(y)
two_body_J2_acc_z = two_body_J2.diff(z)


two_body_J2_acc_x = two_body_J2_acc_x.subs([(r_earth, r_earth_const), (mu, mu_earth), \
                              (J_2, J_2_const)])
two_body_J2_acc_y = two_body_J2_acc_y.subs([(r_earth, r_earth_const), (mu, mu_earth), \
                              (J_2, J_2_const)])
two_body_J2_acc_z = two_body_J2_acc_z.subs([(r_earth, r_earth_const), (mu, mu_earth), \
                              (J_2, J_2_const)])
#print('2 body & J2', two_body_J2_acc_x)

x_acc = two_body_J2_acc_x
y_acc = two_body_J2_acc_y
z_acc = two_body_J2_acc_z

In [163]:
#Add drag to J_2 force equations

x_dot, y_dot, z_dot, x_double_dot, y_double_dot, z_double_dot = \
    sym.symbols('x_dot, y_dot, z_dot, x_double_dot, y_double_dot, z_double_dot')
    
C_D, A, m, density, theta_dot, val, val_dot = \
    sym.symbols('C_D A m density theta_dot val, val_dot')

drag_str = ('-(1/2)*C_D*(A/m)*density*'
                'sqrt((x_dot+theta_dot*y)**2 + (y_dot-theta_dot*x)**2 +'
                'z_dot**2)*(val_dot+theta_dot*val)')
drag_symp = sym.sympify(drag_str)

drag_symp = drag_symp.subs([(A, A_const), (m, m_const), (C_D, C_D_const),\
                        (theta_dot, theta_dot_const)])


x_drag_symp = drag_symp.subs([(r, sym.sqrt(x**2+y**2+z**2)), (val, y), (val_dot, x_dot)])
x_acc = x_acc + x_drag_symp

y_drag_symp = drag_symp.subs([(r, sym.sqrt(x**2+y**2+z**2)), (val, x), (val_dot, y_dot)])
y_acc = y_acc + y_drag_symp

z_drag_symp = drag_symp.subs([(r, sym.sqrt(x**2+y**2+z**2)), (val, z), (val_dot, z_dot)])
z_acc = z_acc + z_drag_symp
    


x_acc_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), x_acc)
y_acc_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), y_acc)
z_acc_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), z_acc)

In [164]:


if (meas_type == 1) or (meas_type == 3):
    
    x_s, y_s, z_s, x_sf, y_sf, z_sf, theta, theta_dot, t, x_dot, y_dot, z_dot = \
        sym.symbols('x_s, y_s, z_s, x_sf, y_sf, z_sf, theta, theta_dot, t, x_dot, y_dot, z_dot')

    #define symbolic rho equation
    rho = ('sqrt((x - x_s)**2 + (y - y_s)**2 + (z - z_s)**2)')
    rho = sym.sympify(rho)
    #sub rotation equation of ecef for eci
    rho = rho.subs(x_s, x_sf*sym.cos(omega_const*t) - y_sf*sym.sin(omega_const*t))
    rho = rho.subs(y_s, x_sf*sym.sin(omega_const*t) + y_sf*sym.cos(omega_const*t))
    rho = rho.subs(z_s, z_sf)

    #define symbolic rho dot equation
    rho_dot = ('(x*x_dot + y*y_dot + z*z_dot - (x_dot*x_s+y_dot*y_s)*cos(theta) + \
               theta_dot*(x*x_s + y*y_s)*sin(theta) + (x_dot*y_s - y_dot*x_s)*sin(theta) +\
               theta_dot*(x*y_s - y*x_s)*cos(theta) - z_dot*z_s)/ rho')
    rho_dot = sym.sympify(rho_dot)
    #execute substitutions for rho_dot
    rho_dot = rho_dot.subs(x_s, x_sf) 
    rho_dot = rho_dot.subs(y_s, y_sf) 
    rho_dot = rho_dot.subs(z_s, z_sf) 
    rho_dot = rho_dot.subs('rho', rho)
    rho_dot = rho_dot.subs(theta, omega_const*t)    
    rho_dot = rho_dot.subs(theta_dot, omega_const)

    rho_fcn = lambdify(((x, y, z, x_sf, y_sf, z_sf, t)), rho)
    rho_dot_fcn = lambdify(((x, y, z, x_dot, y_dot, z_dot, x_sf, y_sf, z_sf, t)), rho_dot)


if (meas_type == 2) or (meas_type == 3):
    
    #x_sf, etc. is the sensor pos in ecef
    #x, y, z is the satellite eci
    x_s, y_s, z_s, x_sf, y_sf, z_sf, theta, x, y, z, t = \
        sym.symbols('x_s, y_s, z_s, x_sf, y_sf, z_sf, theta, x, y, z, t')
    x_L, y_L, z_L, X_L_norm, x_range, y_range, z_range, lon, lat, \
        x_sat_ecef, y_sat_ecef, z_sat_ecef, sen_ecef_norm, omega  = \
        sym.symbols('x_L, y_L, z_L, X_L_norm, x_range, y_range, z_range, lon, lat, \
        x_sat_ecef, y_sat_ecef, z_sat_ecef, sen_ecef_norm, omega')
        

    #define symbolic rho equation
    azimuth = ('atan2(x_L, y_L)') #step 4
    azimuth = sym.sympify(azimuth)
    
    elevation = ('asin(z_L/X_L_norm)') #step 4
    elevation = sym.sympify(elevation)
    elevation = elevation.subs(X_L_norm, sym.sqrt(x_L**2 + y_L**2 + z_L**2))
    
    #step 3
    azimuth = azimuth.subs([(x_L, -x_range*sym.sin(lon) + y_range*sym.cos(lon)), \
            (y_L, -x_range*sym.sin(lat)*sym.cos(lon) - y_range*sym.sin(lat)*sym.sin(lon) + z_range*sym.cos(lat))])
    elevation = elevation.subs([(x_L, -x_range*sym.sin(lon) + y_range*sym.cos(lon)), \
            (y_L, -x_range*sym.sin(lat)*sym.cos(lon) - y_range*sym.sin(lat)*sym.sin(lon) + z_range*sym.cos(lat)), \
            (z_L, x_range*sym.cos(lat)*sym.cos(lon) + y_range*sym.cos(lat)*sym.sin(lon) + z_range*sym.sin(lat))])
    
    #step 2
    azimuth = azimuth.subs([(x_range, x_sat_ecef - x_sf), (y_range, y_sat_ecef - y_sf), \
            (z_range, z_sat_ecef - z_sf), (lat, sym.asin(z_sf/sen_ecef_norm)), (lon, sym.atan2(y_sf, x_sf))])
    elevation = elevation.subs([(x_range, x_sat_ecef - x_sf), (y_range, y_sat_ecef - y_sf), \
            (z_range, z_sat_ecef - z_sf), (lat, sym.asin(z_sf/sen_ecef_norm)), (lon, sym.atan2(y_sf, x_sf))])
    
    #step 1
    azimuth = azimuth.subs([(x_sat_ecef, x*sym.cos(theta) + y*sym.sin(theta)), \
                        (y_sat_ecef, -x*sym.sin(theta) + y*sym.cos(theta)), (z_sat_ecef, z), \
                        (sen_ecef_norm, sym.sqrt(x_sf**2 + y_sf**2 + z_sf**2))])
    elevation = elevation.subs([(x_sat_ecef, x*sym.cos(theta) + y*sym.sin(theta)), \
                        (y_sat_ecef, -x*sym.sin(theta) + y*sym.cos(theta)), (z_sat_ecef, z), \
                        (sen_ecef_norm, sym.sqrt(x_sf**2 + y_sf**2 + z_sf**2))])
    
    
    azimuth = azimuth.subs([(theta, omega_const*t), (omega, omega_const)])
    elevation = elevation.subs([(theta, omega_const*t), (omega, omega_const)])
    
    azimuth_fcn = lambdify(((x, y, z, x_sf, y_sf, z_sf, t)), azimuth)
    elevation_fcn = lambdify(((x, y, z, x_sf, y_sf, z_sf, t)), elevation)
    



In [165]:
#State and A matrix


#define the symbolic state matrix
X = sym.Matrix([x, y, z, x_dot, y_dot, z_dot])
X_dot = sym.Matrix([x_dot, y_dot, z_dot, x_acc, y_acc, z_acc])
    

#partial of the force model (x dot) WRT the state vector
A_mat = X_dot.jacobian(X)
#print(A_mat)

A_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), A_mat)
#print(A_sol_fcn(1,2,3,4,5,6,7,8))

#print(X_dot)
X_dot_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), X_dot)

In [166]:
#define observation model (G) and H_tilde


if meas_type == 1:
    G = sym.Matrix([rho, rho_dot])
    
    
elif meas_type == 2:
    G = sym.Matrix([azimuth, elevation])
    
elif meas_type == 3:
    G = sym.Matrix([azimuth, elevation, rho])

#print(G)
G_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, x_sf, y_sf, z_sf, t), G)

#partial derivitive of observation model WRT the state vector
X_full = X = sym.Matrix([x, y, z, x_dot, y_dot, z_dot, density])
H_tilde = G.jacobian(X_full)
#print(H_tilde)
H_tilde_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, x_sf, y_sf, z_sf, t), H_tilde)

In [167]:



def calc_P_TU(X_mean, ensemble):
    
    dimension = len(X_mean)
    
    P_bar_sum = np.zeros((dimension, dimension))
    
    for ii in range(num_of_ensembles):

        X = ensemble[:, ii].reshape(dimension, 1)
        diff_X = X - X_mean
        P_bar_sum = P_bar_sum + np.dot(diff_X, diff_X.T)

    P_bar = P_bar_sum/(num_of_ensembles-1)

    return P_bar



In [168]:
#define reference state at epoch, covariance at epoch, and R (using measurement noise)

pos_perturbation = 100 * 1e-3 #km
vel_perturbation = .1 * 1e-3 #km/s

density_dimension = num_of_lat * num_of_lon
prob_dimension = 6
fullState_dimension = prob_dimension + density_dimension

X_ref = np.array([truth_xyz[0,0] + pos_perturbation, truth_xyz[0,1] + pos_perturbation, truth_xyz[0,2] \
            + pos_perturbation, truth_xyz[0,3] + vel_perturbation, truth_xyz[0,4] + vel_perturbation, \
            truth_xyz[0,5] + vel_perturbation])

                  
P_sigma_pos = 100 * 1e-3 #km
P_sigma_vel = .1 * 1e-3 #km/s
P_bar_0 = np.diag([P_sigma_pos**2, P_sigma_pos**2, P_sigma_pos**2, \
                   P_sigma_vel**2, P_sigma_vel**2, P_sigma_vel**2, (1e-4)**2]) #(1e-4)**2


(L, V) = np.linalg.eig(P_bar_0[:prob_dimension, :prob_dimension]) #eigenvalues, eigenvectors
Bsqrt_cov = V * np.sqrt(L)
X_RV_ensemble = gen_ensemble(X_ref, Bsqrt_cov, num_of_ensembles)
print(np.shape(X_RV_ensemble))


#for process noise
snc_sigma = 1e-5

       
                  
#Define Measurement Noise
if meas_type == 1:
    sigma_rho = .1 * 1e-3 #km
    sigma_rho_dot = .01 * 1e-3 #km/s
    W = np.array([[1/sigma_rho**2, 0], [0, 1/sigma_rho_dot**2]])
    R = np.linalg.inv(W)
    
elif meas_type == 2:
    arcsec_per_rad = 206264.806247 #arcsec/rad
    noise_rad = (1/arcsec_per_rad)  * 5  #5 arcsec noise (suggested by Moriba for a telescope)
    sigma_az = noise_rad
    sigma_el = noise_rad
    W = np.array([[1/sigma_az**2, 0], [0, 1/sigma_el**2]])
    R = np.linalg.inv(W)
    
elif meas_type == 3:
    arcsec_per_rad = 206264.806247 #arcsec/rad
    noise_rad = (1/arcsec_per_rad)  * 5  #5 arcsec noise (suggested by Moriba for a telescope)
    sigma_az = noise_rad
    sigma_el = noise_rad
    sigma_rho = .1 * 1e-3 #km
    W = np.array([[1/sigma_az**2, 0, 0], [0, 1/sigma_el**2, 0], [0, 0, 1/sigma_rho**2]])
    R = np.linalg.inv(W)
    
    meas_indices = np.array([0, 1, 2])


#reduced rank variable for number of eigvals
q = 39

    
print(X_ref)

(6, 450)
[  1.000000000004150e-01   6.778236300000000e+03   1.000000000000000e-01
  -5.874356678131550e+00   1.000000000003597e-04   4.929354431987092e+00]


In [169]:

#Determine value for q (number of eigenvalues/vecs to keep)

"""

dens_ens = ensembles_of_density_grid.reshape(num_of_ensembles, density_dimension).T

X_density_ensemble = np.zeros((fullState_dimension, num_of_ensembles))
X_density_ensemble[:6, :] = X_RV_ensemble
X_density_ensemble[6:, :] = dens_ens


#mean of X state & density state over ensemble
X_bar = np.sum(X_density_ensemble, axis=1).reshape(fullState_dimension,1)/num_of_ensembles

#calculate Time Updated P
P_TU = calc_P_TU(X_bar, X_density_ensemble)


eigvals, eigvecs = np.linalg.eig(P_TU)

np.shape(eigvecs)


#filename = 'Data Files/Covariance.mat'
#scipy.io.savemat(filename, mdict={'P_bar': P_TU})

indices = np.where(eigvals > 1e-2)[0]
print(np.shape(indices))
#print(indices)

indices = np.where(eigvals > 1e-7)[0]
print(np.shape(indices))
#print(indices)

indices = np.where(eigvals > 1e-8)[0]
print(np.shape(indices))
#print(indices)


eigvec_truncated = eigvecs[:, :449]
eigvals_truncated = eigvals[:449]

L = eigvecs * np.sqrt(eigvals) 

Pppp = np.dot(L, L.T)
#print(np.mean(Pppp - P_TU))


L_tilde = np.append(L, np.ones((2707, 2707))*.5e-1).reshape(2707, 2707+2707)

eigvalss, eigvecss = np.linalg.eig(np.dot(L_tilde, L_tilde.T))
PI = eigvecss[:,:q]

print(np.shape(L_tilde))
print(np.shape(PI))


"""

"\n\ndens_ens = ensembles_of_density_grid.reshape(num_of_ensembles, density_dimension).T\n\nX_density_ensemble = np.zeros((fullState_dimension, num_of_ensembles))\nX_density_ensemble[:6, :] = X_RV_ensemble\nX_density_ensemble[6:, :] = dens_ens\n\n\n#mean of X state & density state over ensemble\nX_bar = np.sum(X_density_ensemble, axis=1).reshape(fullState_dimension,1)/num_of_ensembles\n\n#calculate Time Updated P\nP_TU = calc_P_TU(X_bar, X_density_ensemble)\n\n\neigvals, eigvecs = np.linalg.eig(P_TU)\n\nnp.shape(eigvecs)\n\n\n#filename = 'Data Files/Covariance.mat'\n#scipy.io.savemat(filename, mdict={'P_bar': P_TU})\n\nindices = np.where(eigvals > 1e-2)[0]\nprint(np.shape(indices))\n#print(indices)\n\nindices = np.where(eigvals > 1e-7)[0]\nprint(np.shape(indices))\n#print(indices)\n\nindices = np.where(eigvals > 1e-8)[0]\nprint(np.shape(indices))\n#print(indices)\n\n\neigvec_truncated = eigvecs[:, :449]\neigvals_truncated = eigvals[:449]\n\nL = eigvecs * np.sqrt(eigvals) \n\nPppp = np.

In [170]:
#np.all(np.linalg.eigvals(P_TU) > 0) # but all real

In [ ]:
#KALMAN

def execute_enkf(obs_data, X_RV_ensemble, P, R, X_density_ensemble,\
                         prob_dimension, density_dimension, stop_index):
    
    #initializations
    Q_ECI = np.zeros((6,6))
    
    num_of_meas = np.shape(R)[0]
    X_7elem_mean_updated = np.sum(X_RV_ensemble, axis=1)/num_of_ensembles

    pre_fit_list = np.zeros((stop_index, num_of_meas))
    prefit_bounds_list = np.zeros((stop_index, num_of_meas))
    post_fit_list = np.zeros((stop_index, num_of_meas))

    P_list = np.zeros((stop_index, prob_dimension+1, prob_dimension+1))
    #P_full_list = np.zeros((21, fullState_dimension, fullState_dimension))
    X_7elem_mean_updated_list = np.zeros((stop_index, prob_dimension+1))
    
    density_MSIS_array = np.zeros(stop_index)
    #est_density_grid_array = np.zeros((stop_index, num_of_lat, num_of_lon))

    
    X_distribution = np.zeros((prob_dimension, num_of_ensembles, stop_index))
    density_distribution = np.zeros((num_of_ensembles, stop_index))
    lat_lst_array = np.zeros((2, stop_index))
    X_ensemble = np.zeros((fullState_dimension, num_of_ensembles))

    
    #for Reduced Rank
    dens_ens = X_density_ensemble.reshape(num_of_ensembles, density_dimension).T
    X_ensemble = np.zeros((fullState_dimension, num_of_ensembles))
    X_ensemble[:6, :] = X_RV_ensemble
    X_ensemble[6:, :] = dens_ens

    #mean of X state & density state over ensemble
    X_mean = np.sum(X_ensemble, axis=1).reshape(fullState_dimension,1)/num_of_ensembles
    P = calc_P_TU(X_mean, X_ensemble)
    
    eigvals, eigvecs = np.linalg.eig(P)
    eigvec_truncated = eigvecs[:, :q]
    eigvals_truncated = eigvals[:q]
    L = np.dot(eigvec_truncated, np.diag(np.sqrt(eigvals_truncated)))
    
    P_counter = 0
    
    
    time = obs_data[0, 0]
    obsnum = 0
    
    
    while time < obs_data[stop_index-1,0]:
    #for obsnum in range(len(obs_data[:stop_index])):
    

        #Tracer() ()
        t_init = time - 10 
        if obsnum == 0:
            t_init = time
            
        
        
        
        #determine density to be used in propagation
        (lat_grid_ticks, lst_grid_ticks) = filter_functions.calc_lat_lst_indices(t_init, \
                            X_mean[:3,0], day_of_month_init, hour_init_UT, month_init, \
                                   year_init, omega_const, r_earth_const, lat_res, lon_res)
        
        
        #reduced rank
        epsilon = .95 #number close to 1
        X_mean_wSigma_array = np.tile(X_mean, q) + epsilon*L
        
        
        
        densities = X_density_ensemble[:, lat_grid_ticks, lst_grid_ticks]
        result = X_RV_ensemble

        if (obsnum != 0):
            
            result = np.zeros((prob_dimension, num_of_ensembles))

            for ii in range(num_of_ensembles):
                #set the initial values for the propogator:
                y0 = X_RV_ensemble[:, ii]
                density = densities[ii] * 1e-9
            
                integrator = ode(orbitpropogator_EnKF)
                integrator.set_integrator('dopri5', nsteps=1e6, rtol=1e-12, atol=1e-12)
                integrator.set_f_params(density)
                integrator.set_initial_value(y0, t_init)
                integrator.integrate(time)
                result[:, ii] = integrator.y

        X_RV_ensemble = result.reshape(prob_dimension, num_of_ensembles)
        
        
        #create array that holds X state ensemble & density state ensemble combined
        density_state = X_density_ensemble[:, lat_grid_ticks, lst_grid_ticks]
        X_ensemble[:6, :] = X_RV_ensemble
        X_ensemble[6:, :] = X_density_ensemble.reshape(num_of_ensembles, density_dimension).T 
        
        
        
        #reduced rank
        result = X_mean_wSigma_array[:6,:]
        result_mean = X_mean[:6]
        if (obsnum != 0):
            
            result = np.zeros((prob_dimension, q))

            for ii in range(q):
                y0 = X_mean_wSigma_array[:6, ii]
                density = densities[ii] * 1e-9
            
                integrator = ode(orbitpropogator_EnKF)
                integrator.set_integrator('dopri5', nsteps=1e6, rtol=1e-12, atol=1e-12)
                integrator.set_f_params(density)
                integrator.set_initial_value(y0, t_init)
                integrator.integrate(time)
                result[:, ii] = integrator.y
                
                
            y0 = X_mean[:6]
            density = np.mean(densities) * 1e-9

            integrator = ode(orbitpropogator_EnKF)
            integrator.set_integrator('dopri5', nsteps=1e6, rtol=1e-12, atol=1e-12)
            integrator.set_f_params(density)
            integrator.set_initial_value(y0, t_init)
            integrator.integrate(time)
            result_mean = integrator.y    
            
        
        X_mean_wSigma_array_TU = np.zeros((fullState_dimension, q))
        X_mean_wSigma_array_TU[:6,:] = result.reshape(prob_dimension, q)
        X_mean_wSigma_array_TU[6:,:] = X_mean_wSigma_array[6:,:]
        
        X_mean_TU = np.zeros((fullState_dimension, 1))
        X_mean_TU[:6, 0] = result_mean.reshape(prob_dimension)
        X_mean_TU[6:, 0] = X_mean[6:].reshape(density_dimension)

        L = (1/epsilon) * (X_mean_wSigma_array_TU - np.tile(X_mean_TU, q))
        
        
        #Use SNC P_bar if delta t is less than 100 seconds---------------------------
        if (obs_data[obsnum, 0] != obs_data[obsnum-1, 0]) and (obsnum != 0):
            delta_t = obs_data[obsnum, 0] - obs_data[obsnum-1, 0]
            if obsnum == 0:
                delta_t = 0
            if (delta_t < 15 and snc_flag):
                #build SNC STM
                idenity_6_3 = np.array([(delta_t/2)*np.eye(3), np.eye(3)]).reshape(6, 3)
                snc_stm = delta_t*idenity_6_3
                #Covariance with SNC: Q Matrix
                Q = np.identity(3)*(snc_sigma)**2

                #Q_entire = np.zeros((fullState_dimension, fullState_dimension))
                #Q_entire[:prob_dimension, :prob_dimension] = np.dot(snc_stm, np.dot(Q, snc_stm.T))
                #instead:
                FQ = np.zeros((fullState_dimension, prob_dimension))
                FQ[:prob_dimension, :prob_dimension] = np.dot(snc_stm, np.dot(Q, snc_stm.T))
                
                #build density portion of Q
                #Q_entire[prob_dimension:, prob_dimension:] = np.eye(density_dimension, density_dimension) * 1e-9
                
                #Eq. 24:
                L_tilde = np.zeros((fullState_dimension, q+prob_dimension))
                L_tilde[:, :q] = L
                L_tilde[:, q:] = FQ #Q_entire
                
                #Eq. 25:
                eigvals, X = np.linalg.eig(np.dot(L_tilde.T, L_tilde)) 
                L = np.dot(L_tilde, X)[:, :q]
                

        #Tracer() ()       
        #mean of X state & density state over ensemble, reduced rank, *re-definition of X_mean on purpose*
        ensemble_noise = np.random.randn(num_of_lat * num_of_lon, num_of_ensembles) * 1e-5 #1e-8 #1e-9 #1e-10#
        X_ensemble[prob_dimension:, :] = X_ensemble[prob_dimension:, :] + ensemble_noise
        X_mean = np.sum(X_ensemble, axis=1).reshape(fullState_dimension,1)/num_of_ensembles #Eq. 16
        E = X_ensemble - np.tile(X_mean, num_of_ensembles) #Eq. 17
        
        
        #-------------------Measurement Update--------------------------------------------------------
        
        
        indices = np.where(obs_data[:,0] == time)[0]
        for ii in range(len(indices)):
            obsnum = indices[ii]
            print(time, obsnum)
        
        
            #determine station coordinates for observation eq.
            if int(obs_data[obsnum, 1]) == 1:
                #print('1')
                station_index = 0
                X_s = r_ecef_dss34
            if int(obs_data[obsnum, 1]) == 2:
                #print('2')
                station_index = 1
                X_s = r_ecef_dss65
            if int(obs_data[obsnum, 1]) == 3:
                #print('3')
                station_index = 2
                X_s = r_ecef_dss13
            if int(obs_data[obsnum, 1]) == 4:
                #print('amos')
                station_index = 3
                X_s = r_ecef_diego




            #Tracer() ()
            #assuming Eq. 31 is supposed to be ensemble - x_mean, not just ensemble
            #below is the alternative to Eq. 32 (determined by Penny & I)
            EtransposeE = np.dot(E.T, E)
            eigvals, X = np.linalg.eig(EtransposeE)
            E_perp = np.dot(E, X)[:,q:] #Eq. 32 alternative

            E_doublePrime = np.dot(E, X)[:,:q] #Eq. 35 alternative

            lambdaa = .1 #.2
            P = lambdaa * np.dot(L, L.T) + ((1-lambdaa)/(num_of_ensembles-1)) * np.dot(E_doublePrime, E_doublePrime.T) \
                                    + (1/(num_of_ensembles-1)) * np.dot(E_perp, E_perp.T) #Eq. 37



            H = np.zeros((num_of_meas, fullState_dimension))
            H[:num_of_meas, :prob_dimension+1] = H_tilde_sol_fcn(*X_mean[:6,0], *X_s, time)

            part1 = np.dot(P, H.T)
            part2 = np.dot(H, np.dot(P, H.T))
            K = np.dot(part1, np.linalg.inv(part2 + R)) #Eq. 34

            #computations needed for Eq. 20
            Hx_array = np.zeros((num_of_meas, num_of_ensembles))
            Hx = np.zeros(3)
            for ii in range(num_of_ensembles):

                X = X_RV_ensemble[:, ii]
                Hx = G_sol_fcn(*X, *X_s, time)

                if (meas_type != 1) and (Hx[0] < 0): #if az less than 0, add 2*pi
                    Hx[0] = Hx[0] + 2*math.pi

                Hx_array[:, ii] = Hx.reshape(num_of_meas)

            #Tracer() ()
            #get actual observation
            y_observed = obs_data[obsnum, 2:(2+num_of_meas)].reshape(num_of_meas,1)

            #Eq. 20:
            X_ensemble_updated = np.zeros((fullState_dimension, num_of_ensembles))
            for ii in range(num_of_ensembles):

                e = np.sqrt(np.diag(R)).reshape(num_of_meas, 1) * np.random.randn(num_of_meas, 1)

                correction = np.dot(K, (y_observed + e - Hx_array[:, ii].reshape(num_of_meas, 1)))

                X_member_updated = X_ensemble[:, ii].reshape(fullState_dimension, 1) + correction

                X_ensemble_updated[:, ii] = X_member_updated.reshape(fullState_dimension)



            #update overall density grid ensemble with this lat/lon's updated density values 
            X_density_ensemble = copy.deepcopy(X_ensemble_updated[6:,:].T.reshape(num_of_ensembles, \
                                                                                            num_of_lat, num_of_lon)) 
            X_ensemble = copy.deepcopy(X_ensemble_updated)
            #est_density_grid_array[obsnum] = np.mean(X_density_ensemble, axis=0)

            #Tracer() ()
            #SRF Update equations for updating L for next obs time
            #Eq. 28, using X_mean b/c paper shows mean being propagated then used, not
            #ensemble being propagated and then the mean calculated and used
            Hx = G_sol_fcn(*X_mean[:6,0], *X_s, time)
            X_mean_MU = X_mean + np.dot(K, y - Hx) #not used? Eq. 28


            #Eq. 29:
            H = np.zeros((num_of_meas, fullState_dimension))
            H[:num_of_meas, :prob_dimension+1] = H_tilde_sol_fcn(*X_mean[:6,0], *X_s, time)

            L_tilde = np.zeros((fullState_dimension, q+num_of_meas))
            part1 = np.eye(fullState_dimension, fullState_dimension) - np.dot(K, H)
            L_tilde[:, :q] = np.dot(part1, L)
            L_tilde[:, q:] = np.dot(K, np.sqrt(R)) #Eq. 29

            #Eq. 30
            eigvals, X = np.linalg.eig(np.dot(L_tilde.T, L_tilde))
            L = np.dot(L_tilde, X)[:, :q]




            #save values
            X_7elem_mean_updated = np.zeros((7))
            X_7elem_mean_updated[:6] = np.sum(X_ensemble_updated[:6,:], axis=1)/num_of_ensembles
            X_7elem_mean_updated[6:] = np.sum(X_density_ensemble[:, lat_grid_ticks, lst_grid_ticks])\
                                                                                                /num_of_ensembles
            X_7elem_mean_updated_list[obsnum, :] = X_7elem_mean_updated

            #save values for analysis of distributions and such
            density_distribution[:, obsnum] = X_density_ensemble[:, lat_grid_ticks, lst_grid_ticks] 
            X_distribution[:, :, obsnum] = X_ensemble_updated[:6, :]
            lat_lst_array[:, obsnum] = np.array([latitude_grid[lat_grid_ticks], longitude_grid[lst_grid_ticks]]).reshape(2)




            P_list[obsnum, :6, :6] = P[:6, :6] #is the below supposed to be transposed then reshaped?!?!****
            P_list[obsnum, -1, -1] = P[0,6:].reshape(num_of_lat, num_of_lon)[lat_grid_ticks, lst_grid_ticks]

            #if ((obsnum == 0) or (obsnum % 10 == 0) or (obsnum == 199)):
                #P_full_list[P_counter, :, :] = P
                #P_counter = P_counter + 1


            #-------------RMS / Residuals-------------
            Hx = G_sol_fcn(*X_7elem_mean_updated[:6], *X_s, time)
            if (meas_type != 1) and (Hx[0] < 0): #if az less than 0, add 2*pi
                Hx[0] = Hx[0] + 2*math.pi
            post_fit_resid = y_observed - Hx.reshape(num_of_meas, 1)
            post_fit_list[obsnum, :] = post_fit_resid.reshape(num_of_meas) 


            #pre-fit residual calculation
            #determine observation as evaluated in observation equation with ref. 
            y_G_ref = G_sol_fcn(*X_mean[:6,0], *X_s, time)
            if (meas_type != 1) and (y_G_ref[0] < 0): #if az less than 0, add 2*pi
                    y_G_ref[0] = y_G_ref[0] + 2*math.pi

            #get actual observation
            y_observed = obs_data[obsnum, 2:(2+num_of_meas)].reshape(num_of_meas,1)

            #calculate pre-fit
            prefit_bounds = np.sqrt(np.dot(H, np.dot(P, H.T)) + R)
            prefit_bounds_list[obsnum, :] = np.diag(prefit_bounds)
            prefit = y_observed - y_G_ref
            pre_fit_list[obsnum, :] = prefit.reshape(num_of_meas)


            #Prep for next time step
            X_RV_ensemble = X_ensemble_updated[:6, :]
            
            X_mean = np.sum(X_ensemble, axis=1).reshape(fullState_dimension,1)/num_of_ensembles
            
            
            #save true MSIS density for this time step (which is density at beginning of time used to prop for this step)
            density_MSIS = filter_functions.calc_MSIS_density(time, X_7elem_mean_updated, day_of_year_init, day_of_month_init, \
                                                    hour_init_UT, month_init, year_init, omega_const, r_earth_const)
            density_MSIS = density_MSIS * 1e9  #convert from kg/m**3 to kg/km**3
            density_MSIS_array[obsnum] = density_MSIS

            obsnum = obsnum + 1 #just to ensure that obsnum is not still zero for second time
            
        
        time = time + 10


    
    return (X_7elem_mean_updated_list, P_list, pre_fit_list, prefit_bounds_list, post_fit_list, density_MSIS_array, \
            X_distribution, density_distribution, lat_lst_array, X_density_ensemble, X_ensemble)
            
            #, est_density_grid_array)

In [ ]:
#Call EnKF

snc_flag = True
stop_index = len(measurement_array) #558 * 3 #92.56 minutes #/2

(X_mean_updated_list_EnKF, P_list_EnKF, pre_fit_list, prefit_bounds_list, post_fit_list_EnKF, \
 density_MSIS_array, X_distribution, density_distribution, lat_lst_array, density_state_ensemble, \
                     X_ensemble) = \
            execute_enkf(measurement_array, X_RV_ensemble, P_bar_0, R, ensembles_of_density_grid,\
                         prob_dimension, density_dimension, stop_index)
    
#, est_density_grid_array) = \

0.0 0
30.0 1
60.0 2
90.0 3
120.0 4
150.0 5
180.0 6
210.0 7
240.0 8
270.0 9
300.0 10
330.0 11
360.0 12
390.0 13
420.0 14
450.0 15
480.0 16
510.0 17
540.0 18
570.0 19
600.0 20
630.0 21
660.0 22
690.0 23
720.0 24
750.0 25
780.0 26
810.0 27
840.0 28
870.0 29
900.0 30
930.0 31
960.0 32
990.0 33
1020.0 34
1050.0 35
1080.0 36
1110.0 37
1140.0 38
1170.0 39
1200.0 40
1230.0 41
1260.0 42
1290.0 43
1320.0 44
1350.0 45
1380.0 46
1410.0 47
1440.0 48
1470.0 49
1500.0 50
1530.0 51
1560.0 52
1590.0 53
1620.0 54
1650.0 55
1680.0 56
1710.0 57
1740.0 58
1770.0 59
1800.0 60
1830.0 61
1860.0 62
1890.0 63
1920.0 64
1950.0 65
1980.0 66
2010.0 67
2040.0 68
2070.0 69
2100.0 70
2130.0 71
2160.0 72
2190.0 73
2220.0 74
2250.0 75
2280.0 76
2310.0 77
2340.0 78
2370.0 79
2400.0 80
2430.0 81
2460.0 82
2490.0 83
2520.0 84
2550.0 85
2580.0 86
2610.0 87
2640.0 88
2670.0 89
2700.0 90
2730.0 91
2760.0 92
2790.0 93
2820.0 94
2850.0 95
2880.0 96
2910.0 97
2940.0 98
2970.0 99
3000.0 100
3030.0 101
3060.0 102
3090.0 103
3120.

21690.0 723
21720.0 724
21750.0 725
21780.0 726
21810.0 727
21840.0 728
21870.0 729
21900.0 730
21930.0 731
21960.0 732
21990.0 733
22020.0 734
22050.0 735
22080.0 736
22110.0 737
22140.0 738
22170.0 739
22200.0 740
22230.0 741
22260.0 742
22290.0 743
22320.0 744
22350.0 745
22380.0 746
22410.0 747
22440.0 748
22470.0 749
22500.0 750
22530.0 751
22560.0 752
22590.0 753
22620.0 754
22650.0 755
22680.0 756
22710.0 757
22740.0 758
22770.0 759
22800.0 760
22830.0 761
22860.0 762
22890.0 763
22920.0 764
22950.0 765
22980.0 766
23010.0 767
23040.0 768
23070.0 769
23100.0 770
23130.0 771
23160.0 772
23190.0 773
23220.0 774
23250.0 775
23280.0 776
23310.0 777
23340.0 778
23370.0 779
23400.0 780
23430.0 781
23460.0 782
23490.0 783
23520.0 784
23550.0 785
23580.0 786
23610.0 787
23640.0 788
23670.0 789
23700.0 790
23730.0 791
23760.0 792
23790.0 793
23820.0 794
23850.0 795
23880.0 796
23910.0 797
23940.0 798
23970.0 799
24000.0 800
24030.0 801
24060.0 802
24090.0 803
24120.0 804
24150.0 805
2418

40980.0 1375
41010.0 1376
41040.0 1377
41070.0 1378
41100.0 1379
41130.0 1380
41160.0 1381
41190.0 1382
41220.0 1383
41250.0 1384
41280.0 1385
41310.0 1386
41340.0 1387
41370.0 1388
41400.0 1389
41430.0 1390
41460.0 1391
41490.0 1392
41520.0 1393
41550.0 1394
41580.0 1395
41610.0 1396
41640.0 1397
41670.0 1398
41700.0 1399
41730.0 1400
41760.0 1401
41790.0 1402
41820.0 1403
41850.0 1404
41880.0 1405
41910.0 1406
41940.0 1407
41970.0 1408
42000.0 1409
42030.0 1410
42060.0 1411
42090.0 1412
42120.0 1413
42150.0 1414
42180.0 1415
42210.0 1416
42240.0 1417
42270.0 1418
42300.0 1419
42330.0 1420
42360.0 1421
42360.0 1422
42390.0 1423
42390.0 1424
42420.0 1425
42420.0 1426
42450.0 1427
42450.0 1428
42480.0 1429
42480.0 1430
42510.0 1431
42510.0 1432
42540.0 1433
42540.0 1434
42570.0 1435
42570.0 1436
42600.0 1437
42600.0 1438
42630.0 1439
42630.0 1440
42660.0 1441
42660.0 1442
42690.0 1443
42690.0 1444
42720.0 1445
42720.0 1446
42750.0 1447
42780.0 1448
42810.0 1449
42840.0 1450
42870.0 1451

58770.0 2006
58800.0 2007
58830.0 2008
58860.0 2009
58890.0 2010
58920.0 2011
58950.0 2012
58980.0 2013
59010.0 2014
59040.0 2015
59070.0 2016
59100.0 2017
59130.0 2018
59160.0 2019
59190.0 2020
59220.0 2021
59250.0 2022
59280.0 2023
59310.0 2024
59340.0 2025
59370.0 2026
59400.0 2027
59430.0 2028
59460.0 2029
59490.0 2030
59520.0 2031
59550.0 2032
59580.0 2033
59610.0 2034
59640.0 2035
59670.0 2036
59700.0 2037
59730.0 2038
59760.0 2039
59790.0 2040
59820.0 2041
59850.0 2042
59880.0 2043
59880.0 2044
59910.0 2045
59910.0 2046
59940.0 2047
59940.0 2048
59970.0 2049
59970.0 2050
60000.0 2051
60000.0 2052
60030.0 2053
60030.0 2054
60060.0 2055
60060.0 2056
60090.0 2057
60090.0 2058
60120.0 2059
60120.0 2060
60150.0 2061
60150.0 2062
60180.0 2063
60180.0 2064
60210.0 2065
60210.0 2066
60240.0 2067
60270.0 2068
60300.0 2069
60330.0 2070
60360.0 2071
60390.0 2072
60420.0 2073
60450.0 2074
60480.0 2075
60510.0 2076
60540.0 2077
60570.0 2078
60600.0 2079
60630.0 2080
60660.0 2081
60690.0 2082

77340.0 2637
77370.0 2638
77400.0 2639
77430.0 2640
77460.0 2641
77490.0 2642
77520.0 2643
77550.0 2644
77580.0 2645
77610.0 2646
77640.0 2647
77670.0 2648
77700.0 2649
77730.0 2650
77760.0 2651
77790.0 2652
77820.0 2653
77850.0 2654
77880.0 2655
77910.0 2656
77940.0 2657
77970.0 2658
78000.0 2659
78030.0 2660
78060.0 2661
78090.0 2662
78120.0 2663
78150.0 2664
78180.0 2665
78210.0 2666
78240.0 2667
78270.0 2668
78300.0 2669
78330.0 2670
78360.0 2671
78390.0 2672
78420.0 2673
78450.0 2674
78480.0 2675
78510.0 2676
78540.0 2677
78570.0 2678
78600.0 2679
78630.0 2680
78660.0 2681
78690.0 2682
78720.0 2683
78750.0 2684
78780.0 2685
78810.0 2686
78840.0 2687
78870.0 2688
78900.0 2689
78930.0 2690
78960.0 2691
78990.0 2692
79020.0 2693
79050.0 2694
79080.0 2695
79110.0 2696
79140.0 2697
79170.0 2698
79200.0 2699
79230.0 2700
79260.0 2701
79290.0 2702
79320.0 2703
79350.0 2704
79380.0 2705
79410.0 2706
79440.0 2707
79470.0 2708
79500.0 2709
79530.0 2710
79560.0 2711
79590.0 2712
79620.0 2713

96270.0 3268
96300.0 3269
96330.0 3270
96360.0 3271
96390.0 3272
96420.0 3273
96450.0 3274
96480.0 3275
96510.0 3276
96540.0 3277
96570.0 3278
96600.0 3279
96630.0 3280
96660.0 3281
96690.0 3282
96720.0 3283
96750.0 3284
96780.0 3285
96810.0 3286
96840.0 3287
96870.0 3288
96900.0 3289
96930.0 3290
96960.0 3291
96990.0 3292
97020.0 3293
97050.0 3294
97080.0 3295
97110.0 3296
97140.0 3297
97170.0 3298
97200.0 3299
97230.0 3300
97260.0 3301
97290.0 3302
97320.0 3303
97350.0 3304
97380.0 3305
97410.0 3306
97440.0 3307
97470.0 3308
97500.0 3309
97530.0 3310
97560.0 3311
97590.0 3312
97620.0 3313
97650.0 3314
97680.0 3315
97710.0 3316
97740.0 3317
97770.0 3318
97800.0 3319
97830.0 3320
97860.0 3321
97890.0 3322
97920.0 3323
97950.0 3324
97980.0 3325
98010.0 3326
98040.0 3327
98070.0 3328
98100.0 3329
98130.0 3330
98160.0 3331
98190.0 3332
98220.0 3333
98250.0 3334
98280.0 3335
98310.0 3336
98340.0 3337
98370.0 3338
98400.0 3339
98430.0 3340
98460.0 3341
98490.0 3342
98520.0 3343
98550.0 3344

114120.0 3863
114150.0 3864
114180.0 3865
114210.0 3866
114240.0 3867
114270.0 3868
114300.0 3869
114330.0 3870
114360.0 3871
114390.0 3872
114420.0 3873
114450.0 3874
114480.0 3875
114510.0 3876
114540.0 3877
114570.0 3878
114600.0 3879
114630.0 3880
114660.0 3881
114690.0 3882
114720.0 3883
114750.0 3884
114780.0 3885
114810.0 3886
114840.0 3887
114870.0 3888
114900.0 3889
114930.0 3890
114960.0 3891
114990.0 3892
115020.0 3893
115050.0 3894
115080.0 3895
115110.0 3896
115140.0 3897
115170.0 3898
115200.0 3899
115230.0 3900
115260.0 3901
115290.0 3902
115320.0 3903
115350.0 3904
115380.0 3905
115410.0 3906
115440.0 3907
115470.0 3908
115500.0 3909
115530.0 3910
115560.0 3911
115590.0 3912
115620.0 3913
115650.0 3914
115680.0 3915
115710.0 3916
115740.0 3917
115770.0 3918
115800.0 3919
115830.0 3920
115860.0 3921
115890.0 3922
115920.0 3923
115950.0 3924
115980.0 3925
116010.0 3926
116040.0 3927
116070.0 3928
116100.0 3929
116130.0 3930
116160.0 3931
116190.0 3932
116220.0 3933
116250

131160.0 4449
131190.0 4450
131190.0 4451
131220.0 4452
131220.0 4453
131250.0 4454
131250.0 4455
131280.0 4456
131280.0 4457
131310.0 4458
131310.0 4459
131340.0 4460
131340.0 4461
131370.0 4462
131400.0 4463
131430.0 4464
131460.0 4465
131490.0 4466
131520.0 4467
131550.0 4468
131580.0 4469
131610.0 4470
131640.0 4471
131670.0 4472
131700.0 4473
131730.0 4474
131760.0 4475
131790.0 4476
131820.0 4477
131850.0 4478
131880.0 4479
131910.0 4480
131940.0 4481
131970.0 4482
132000.0 4483
132030.0 4484
132060.0 4485
132090.0 4486
132120.0 4487
132150.0 4488
132180.0 4489
132210.0 4490
132240.0 4491
132270.0 4492
132300.0 4493
132330.0 4494
132360.0 4495
132390.0 4496
132420.0 4497
132450.0 4498
132480.0 4499
132510.0 4500
132540.0 4501
132570.0 4502
132600.0 4503
132630.0 4504
132660.0 4505
132690.0 4506
132720.0 4507
132750.0 4508
132780.0 4509
132810.0 4510
132840.0 4511
132870.0 4512
132900.0 4513
132930.0 4514
132960.0 4515
132990.0 4516
133020.0 4517
133050.0 4518
133080.0 4519
133110

147840.0 5035
147870.0 5036
147900.0 5037
147930.0 5038
147960.0 5039
147990.0 5040
148020.0 5041
148050.0 5042
148080.0 5043
148110.0 5044
148140.0 5045
148170.0 5046
148200.0 5047
148230.0 5048
148260.0 5049
148290.0 5050
148320.0 5051
148350.0 5052
148380.0 5053
148410.0 5054
148440.0 5055
148470.0 5056
148500.0 5057
148530.0 5058
148530.0 5059
148560.0 5060
148560.0 5061
148590.0 5062
148590.0 5063
148620.0 5064
148620.0 5065
148650.0 5066
148650.0 5067
148680.0 5068
148710.0 5069
148740.0 5070
148770.0 5071
148800.0 5072
148830.0 5073
148860.0 5074
148890.0 5075
148920.0 5076
148950.0 5077
148980.0 5078
149010.0 5079
149040.0 5080
149070.0 5081
149100.0 5082
149130.0 5083
149160.0 5084
149190.0 5085
149220.0 5086
149250.0 5087
149280.0 5088
149310.0 5089
149340.0 5090
149370.0 5091
149400.0 5092
149430.0 5093
149460.0 5094
149490.0 5095
149520.0 5096
149550.0 5097
149580.0 5098
149610.0 5099
149640.0 5100
149670.0 5101
149700.0 5102
149730.0 5103
149760.0 5104
149790.0 5105
149820

In [ ]:
saveFig_bool = True

save_results = False

#stop_index = 4

In [ ]:
#Save Results



saveFig_bool = True

save_results = False

save_density_grid = False #save time series of density grid, True if saved the est. density grid within EnKF

"""

if ((save_density_grid == False) and (save_results)):

    #generate density grid for final time in order to compare entire final density grid estimate to truth
    alt = np.linalg.norm(X_mean_updated_list_EnKF[-1,:3]) - r_earth_const

    final_density_grid_truth = filter_functions.gen_one_ensemble(latitude_grid, longitude_grid, alt,\
                                                        day_of_year_init, measurement_array[stop_index-1,0]) #final time
   
    mydict = {'X_mean_updated_list_EnKF': X_mean_updated_list_EnKF, 'P_list_EnKF': P_list_EnKF, \
          'post_fit_list_EnKF': post_fit_list_EnKF, 'density_MSIS_array': density_MSIS_array, \
          'X_distribution': X_distribution, \#'P_full_list': P_full_list, \
          'density_distribution': density_distribution, 'lat_lst_array': lat_lst_array,
         'final_density_ensemble_est': density_state_ensemble, 'final_X_ensemble': X_ensemble,\
        'true_density_array': true_density_array, \
          'final_density_grid_truth': final_density_grid_truth*1e9, 'measurement_array': measurement_array, \
             'truth_xyz': truth_xyz, 'lat_lst_meas_array': lat_lst_meas_array}


    output = open('Figures/Results.pkl', 'wb')
    pickle.dump(mydict, output)
    output.close()


    #MATLAB file

    filename = 'Data Files/Results.mat'

    import scipy.io
    scipy.io.savemat(filename, mdict={'X_mean_updated_list_EnKF': X_mean_updated_list_EnKF, 'P_list_EnKF': P_list_EnKF, \
              'post_fit_list_EnKF': post_fit_list_EnKF, 'density_MSIS_array': density_MSIS_array, \
              'X_distribution': X_distribution, \#'P_full_list': P_full_list, \
              'density_distribution': density_distribution, 'lat_lst_array': lat_lst_array,
             'final_density_ensemble_est': density_state_ensemble, 'final_X_ensemble': X_ensemble,
            'true_density_array': true_density_array, 
            'final_density_grid_truth': final_density_grid_truth*1e9, 'measurement_array': measurement_array, \
             'truth_xyz': truth_xyz, 'lat_lst_meas_array': lat_lst_meas_array})
    

    
    
if (save_density_grid and save_results):

    final_density_grid_truth_timeSeries = np.zeros((stop_index, num_of_lat, num_of_lon))    

    for ii in range(stop_index):

        #generate density grid for final time in order to compare entire final density grid estimate to truth
        alt = np.linalg.norm(X_mean_updated_list_EnKF[ii,:3]) - r_earth_const
        final_density_grid_truth_timeSeries[ii] = filter_functions.gen_one_ensemble(latitude_grid, longitude_grid, alt,\
                                                    day_of_year_init, measurement_array[ii,0]) #final time

    #PKL file

    mydict = {'X_mean_updated_list_EnKF': X_mean_updated_list_EnKF, 'P_list_EnKF': P_list_EnKF, \
              'post_fit_list_EnKF': post_fit_list_EnKF, 'density_MSIS_array': density_MSIS_array, \
              'X_distribution': X_distribution, \#'P_full_list': P_full_list,  \
              'density_distribution': density_distribution, 'lat_lst_array': lat_lst_array,
             'final_density_ensemble_est': density_state_ensemble, 'final_X_ensemble': X_ensemble,\
            'true_density_array': true_density_array, \
              'final_density_grid_truth_timeSeries': final_density_grid_truth_timeSeries*1e9,
             'est_density_grid_array': est_density_grid_array, 'measurement_array': measurement_array, \
             'truth_xyz': truth_xyz, 'lat_lst_meas_array': lat_lst_meas_array}

    output = open('Figures/Results.pkl', 'wb')
    pickle.dump(mydict, output)
    output.close()



    #MATLAB file

    filename = 'Data Files/Results.mat'

    import scipy.io
    scipy.io.savemat(filename, mdict={'X_mean_updated_list_EnKF': X_mean_updated_list_EnKF, 'P_list_EnKF': P_list_EnKF, \
              'post_fit_list_EnKF': post_fit_list_EnKF, 'density_MSIS_array': density_MSIS_array, \
              'X_distribution': X_distribution, \#'P_full_list': P_full_list,  \
              'density_distribution': density_distribution, 'lat_lst_array': lat_lst_array,
             'final_density_ensemble_est': density_state_ensemble, 'final_X_ensemble': X_ensemble,
            'true_density_array': true_density_array, 
            'final_density_grid_truth_timeSeries': final_density_grid_truth_timeSeries*1e9,
            'est_density_grid_array': est_density_grid_array, 'measurement_array': measurement_array, \
             'truth_xyz': truth_xyz, 'lat_lst_meas_array': lat_lst_meas_array})

    """


In [ ]:
#Read Data (from saved run) in to Generate Results

"""

# read python dict containing densities
results_file = open('Figures/12Periods_trulyObs_0RAAN/Results.pkl', 'rb') #Results_1Period/Results_558.pkl
mydict = pickle.load(results_file)
results_file.close()

X_mean_updated_list_EnKF = mydict['X_mean_updated_list_EnKF'] 
P_list_EnKF = mydict['P_list_EnKF'] 
post_fit_list_EnKF = mydict['post_fit_list_EnKF'] 
density_MSIS_array = mydict['density_MSIS_array'] 
X_distribution = mydict['X_distribution'] 
density_distribution = mydict['density_distribution'] 
lat_lst_array = mydict['lat_lst_array']
final_density_ensemble_est = mydict['final_density_ensemble_est']
final_X_ensemble = mydict['final_X_ensemble']
true_density_array = mydict['true_density_array']
final_density_grid_truth_timeSeries = mydict['final_density_grid_truth_timeSeries']
est_density_grid_array = mydict['est_density_grid_array']

                

"""

In [ ]:
stop_index = stop_index -1
times = measurement_array[:stop_index, 0]/(60)

time_str = 'Time (minutes)'

time_repeated_ensemble = np.repeat(times[:stop_index], num_of_ensembles)
time_repeated_X = np.repeat(times[:stop_index], num_of_ensembles)

#"""
#calculate the normed density distribution by subtracting the mean of the ensemble from each ensemble   
#density_distribution shape = num_of_ensembles x stop_index
density_distribution_mean = np.mean(density_distribution[:,:stop_index], axis=0).reshape(1, stop_index)

density_distribution_mean_tiled = np.tile(density_distribution_mean, (1, num_of_ensembles))
density_distribution_mean_tiled = density_distribution_mean_tiled.reshape(num_of_ensembles, stop_index)

density_distribution_normed = density_distribution[:, :stop_index] - density_distribution_mean_tiled

fig, ax = plt.subplots()
ax.semilogy(time_repeated_ensemble, density_distribution_normed.T.flatten())
#plt.scatter(time_repeated_ensemble, density_distribution_normed.T.flatten())
plt.ylabel(r'Distribution $(kg/km^3)$', fontsize=18)
plt.xlabel(
time_str, fontsize=18)
#plt.ylim(-.5e-3, .5e-3)
plt.title('Density Ensemble Distribution (normed by mean)', fontsize=18)
plt.show()



#calculate the normed density distribution by subtracting the truth of the ensemble from each ensemble   
#density_distribution shape = num_of_ensembles x stop_index
density_distribution_truth = true_density_array[:stop_index].reshape(1, stop_index)
density_distribution_normed = density_distribution[:, :stop_index] - \
                    np.tile(density_distribution_truth, (num_of_ensembles, 1))

fig_dens_dist, ax = plt.subplots()
#ax.semilogy(time_repeated_ensemble, density_distribution_normed.T.flatten())
plt.scatter(time_repeated_ensemble, density_distribution_normed.T.flatten())
plt.ylabel(r'Distribution $(kg/km^3)$', fontsize=18)
plt.xlabel(time_str, fontsize=18)
plt.ylim(-1e-2, 1e-2)
plt.title('Density Ensemble Distribution (normed by truth)', fontsize=18)
plt.show()
#"""

#calculate the normed X pos distribution by subtracting the mean of the ensemble from each ensemble   
#X_distribution shape = 6 x num_of_ensembles x stop_index
X_distribution_Xpos = X_distribution[0,:,:stop_index]
X_distribution_Xpos_mean = np.mean(X_distribution_Xpos, axis=0).reshape(1, stop_index)
X_distribution_Xpos_mean_tiled = np.tile(X_distribution_Xpos_mean, (1, num_of_ensembles))
X_distribution_Xpos_mean_tiled = X_distribution_Xpos_mean_tiled.reshape(num_of_ensembles, stop_index)
X_distribution_Xpos_diff = X_distribution_Xpos - X_distribution_Xpos_mean_tiled

fig = plt.figure()
plt.scatter(time_repeated_X, X_distribution_Xpos_diff.T.flatten())
plt.ylabel(r'Distribution (km)', fontsize=18)
plt.xlabel(time_str, fontsize=18)
#plt.ylim(-1e-4,1e-4)
plt.title('State X Position Ensemble Distribution (normed by mean)', fontsize=18)
plt.show()



#calculate the normed X pos distribution by subtracting the truth of the ensemble from each ensemble   
#X_distribution shape = 6 x num_of_ensembles x stop_index
X_distribution_Xpos_truth = truth_xyz[:stop_index, 0].reshape(1, stop_index)
X_distribution_Xpos_truth_normed = X_distribution_Xpos - np.tile(X_distribution_Xpos_truth, (num_of_ensembles, 1))

fig_X_dist = plt.figure()
plt.scatter(time_repeated_ensemble, X_distribution_Xpos_truth_normed.T.flatten())
plt.ylabel(r'Distribution (km)', fontsize=18)
plt.xlabel(time_str, fontsize=18)
#plt.ylim(-1e-4, 1e-4)
plt.title('State X Position Ensemble Distribution (normed by truth)', fontsize=18)
plt.show()



if saveFig_bool:
    fig_dens_dist.savefig('Figures/dens_dist.png')
    fig_X_dist.savefig('Figures/fig_X_dist.png')


In [ ]:


time_str = 'Time (minutes)'


times = measurement_array[:stop_index, 0]/(60)

time_repeated_ensemble = np.repeat(times[:stop_index], num_of_ensembles)
time_repeated_X = np.repeat(times[:stop_index], num_of_ensembles)


indices_1 = np.where(measurement_array[:stop_index, 1] == 1)[0]
indices_2 = np.where(measurement_array[:stop_index, 1] == 2)[0]
indices_3 = np.where(measurement_array[:stop_index, 1] == 3)[0]
indices_4 = np.where(measurement_array[:stop_index, 1] == 4)[0]


fig_lat = plt.figure()
plt.scatter(times, np.degrees(lat_lst_meas_array[:stop_index, 0]), s=70, c='b',marker='+')
plt.scatter(times, np.degrees(lat_lst_array[0,:stop_index]), s=70, c='c',marker='+')
plt.ylabel('Degrees', fontsize=18)
plt.xlabel(time_str, fontsize=18)
plt.title('Latitude', fontsize=18)
legend_names = ['Truth', 'EnKF']
plt.legend(legend_names, fontsize=16)
plt.show()



fig_LST = plt.figure()
plt.scatter(times, np.degrees(lat_lst_meas_array[:stop_index, 1]), s=70, c='b', marker='+')
plt.scatter(times, np.degrees(lat_lst_array[1,:stop_index]), s=70, c='c', marker='+')
plt.ylabel('Degrees', fontsize=18)
plt.xlabel(time_str, fontsize=18)
plt.title('LST', fontsize=18)
legend_names = ['Truth', 'EnKF']
plt.legend(legend_names, fontsize=16)
plt.show()


fig_density_comparison = plt.figure()
plt.scatter(times, density_MSIS_array[:stop_index], s=70, c='c', marker='+')
plt.scatter(times, true_density_array[:stop_index], s=70, c='b', marker='+')
#plt.ylim([-1e-11,1e-11])
plt.ylabel(r'$kg/km^3$', fontsize=18)
plt.xlabel(time_str, fontsize=18)
plt.ylim([0,2e-2])
plt.title('MSIS Filter Density v. True Density', fontsize=18)
legend_names = ['MSIS', 'Meas Truth']
plt.legend(legend_names, fontsize=16)
plt.show()


fig_density_comparison = plt.figure()
plt.scatter(times, density_MSIS_array[:stop_index], s=70, c='c', marker='+')
plt.scatter(times, X_mean_updated_list_EnKF[:stop_index, 6], s=70, c='b', marker='+')
#plt.ylim([-1e-11,1e-11])
plt.ylabel(r'$kg/km^3$', fontsize=18)
plt.xlabel(time_str, fontsize=18)
plt.ylim([0,2e-2])
plt.title('Estimated v. MSIS Filter Density', fontsize=18)
legend_names = ['MSIS', 'Estimated']
plt.legend(legend_names, fontsize=16)
plt.show()




fig_density_comparison = plt.figure()
plt.scatter(times, true_density_array[:stop_index], s=70, c='b', marker='+')
plt.scatter(times, X_mean_updated_list_EnKF[:stop_index, 6], s=70, c='c', marker='+')
#plt.ylim([-1e-11,1e-11])
plt.ylabel(r'$kg/km^3$', fontsize=18)
plt.xlabel(time_str, fontsize=18)
plt.ylim([0,2e-2])
plt.title('Estimated v. True Density', fontsize=18)
legend_names = ['True', 'Estimated']
plt.legend(legend_names, fontsize=16)
plt.show()



perc_error = 100 * np.absolute(X_mean_updated_list_EnKF[:stop_index, 6] - true_density_array[:stop_index])/true_density_array[:stop_index]

fig_percent = plt.figure()
plt.scatter(times[indices_1], perc_error[indices_1], s=50, c='m', marker='s')
plt.scatter(times[indices_2], perc_error[indices_2], s=50, c='g', marker='^')
plt.scatter(times[indices_3], perc_error[indices_3], s=50, c='r', marker='D')
plt.scatter(times[indices_4], perc_error[indices_4], s=50, c='k', marker='o')
#plt.ylim([-1e-11,1e-11])
plt.ylabel('Percent', fontsize=18)
legend_names = ['Station 1', 'Station 2', 'Station 3', 'Station 4']
#plt.legend(legend_names, fontsize=10)
plt.xlabel(time_str, fontsize=18)
plt.title('Percent Error of Estimated Density', fontsize=18)
plt.show()



mean = np.mean(X_mean_updated_list_EnKF[:stop_index, 6])
est_density_array_normalized = X_mean_updated_list_EnKF[:stop_index, 6] - mean

fig_est = plt.figure()
plt.scatter(times, est_density_array_normalized[:stop_index], s=70, c='b', marker='+')
plt.ylabel(r'$kg/km^3$', fontsize=18)
plt.xlabel(time_str, fontsize=18)
plt.title('Normalized Estimated Density', fontsize=18)
plt.show()



if saveFig_bool:
    fig_lat.savefig('Figures/latitude.png')
    fig_LST.savefig('Figures/LST.png')
    fig_density_comparison.savefig('Figures/density_comparison.png')
    fig_percent.savefig('Figures/percent.png')
    fig_est.savefig('Figures/est_density.png')


In [ ]:
np.where(perc_error < .1)

In [ ]:
# Rank Histograms


#Density

rank_array = np.zeros((stop_index, 1))

for ii in range(stop_index):
    
    new_array = np.append(density_distribution[:, ii], true_density_array[ii])
    
    rank = scipy.stats.rankdata(new_array, method='min')[-1]
    
    rank_array[ii] = rank
    
indices = np.where(rank_array == 1)[0]
indices1 = np.where(rank_array == 450)[0]
print(times[indices1])
print(len(indices))


fig_rank_hist = plt.figure()
plt.hist(rank_array, bins=451)
plt.ylabel('Frequency', fontsize=18)
plt.xlabel('Rank', fontsize=18)
plt.title('Density Ensemble Rank Histogram', fontsize=18)
plt.xlim([0,451])
plt.show()

fig_rank_hist.savefig('Figures/rank_hist.png')




In [ ]:
rank_array

In [ ]:
def calc_display_results_pre(pre_fit_list, prefit_bounds_list, measurement_array, R, meas_type, stop_index, saveFig_bool, time_str):
    
    
    rms_1 = 'Range ='
    unit_1 = 'km'
    ylabel_1 = 'Range Residuals (km)'
    title_1 = 'Range pre-fit Residuals'
    save_fig_1 = 'prefit_range.png'
    rms_2 = 'Range Rate ='
    unit_2 = 'km/s'
    ylabel_2 = 'Range Rate Residuals (km/s)'
    title_2 = 'Range Rate pre-fit Residuals'
    save_fig_2 = 'prefit_rangeRate.png'
    pre_fit_list_new = copy.deepcopy(pre_fit_list)
         
    if (meas_type == 2) or (meas_type == 3):
        rms_1 = 'Azimuth ='
        unit_1 = 'degrees'
        rms_2 = 'Elevation ='
        unit_2 = 'degrees'
        pre_fit_list_new[:, 0] = np.degrees(pre_fit_list[:, 0])
        pre_fit_list_new[:, 1] = np.degrees(pre_fit_list[:, 1])
        ylabel_1 = 'Azimuth Residuals (degrees)'
        title_1 = 'Azimuth pre-fit Residuals'
        save_fig_1 = 'prefit_az.png'
        ylabel_2 = 'Elevation Residuals (degrees)'
        title_2 = 'Elevation pre-fit Residuals'
        save_fig_2 = 'prefit_el_rate.png'

        
    

    times = measurement_array[:stop_index,0]/(60)
    
    indices_1 = np.where(measurement_array[:stop_index, 1] == 1)[0]
    indices_2 = np.where(measurement_array[:stop_index, 1] == 2)[0]
    indices_3 = np.where(measurement_array[:stop_index, 1] == 3)[0]
    indices_4 = np.where(measurement_array[:stop_index, 1] == 4)[0]

    
    
    #pre-fit
    print('pre-fit RMS:')
    pre_fit_1_list_4RMS = pre_fit_list_new[:, 0]
    prefit_1_rms = np.sqrt(np.mean(np.square(pre_fit_1_list_4RMS)))
    print(rms_1, "%.4f" % prefit_1_rms, unit_1)

    pre_fit_2_list_4RMS = pre_fit_list_new[:, 1]
    prefit_2_rms = np.sqrt(np.mean(np.square(pre_fit_2_list_4RMS)))
    print(rms_2, "%.4f" % prefit_2_rms, unit_2)
    
    if meas_type == 3:
        pre_fit_3_list_4RMS = pre_fit_list_new[:, 2]
        prefit_3_rms = np.sqrt(np.mean(np.square(pre_fit_3_list_4RMS)))
        print('Range =', "%.3f" % prefit_3_rms, 'km')
    
    
    covar_env_upper1 = np.degrees(abs(prefit_bounds_list[:stop_index, 0]))*3
    covar_env_upper2 = np.degrees(abs(prefit_bounds_list[:stop_index, 1]))*3

   
    #pre-fit Residuals
    fig_preFit_az = plt.figure()
    plt.plot(times, covar_env_upper1, label='_nolegend_', c='g')
    plt.plot(times, -covar_env_upper1, label='_nolegend_', c='g')
    plt.scatter(times[indices_1], pre_fit_list_new[indices_1, 0], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], pre_fit_list_new[indices_2, 0], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], pre_fit_list_new[indices_3, 0], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], pre_fit_list_new[indices_4, 0], s=50, c='k', marker='o')
    plt.ylabel(ylabel_1, fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.title(title_1, fontsize=18)
    legend_names = ['Station 1', 'Station 2']
    plt.legend(legend_names, fontsize=10)
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.tight_layout()
    plt.show()
    #fig.savefig(save_fig_1)

    fig_preFit_el = plt.figure()
    plt.plot(times, covar_env_upper2, label='_nolegend_', c='g')
    plt.plot(times, -covar_env_upper2, label='_nolegend_', c='g')
    plt.scatter(times[indices_1], pre_fit_list_new[indices_1, 1], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], pre_fit_list_new[indices_2, 1], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], pre_fit_list_new[indices_3, 1], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], pre_fit_list_new[indices_4, 1], s=50, c='k', marker='o')
    plt.ylabel(ylabel_2, fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.title(title_2, fontsize=18)
    plt.legend(legend_names, fontsize=10)
    #plt.ylim([-.01,.01])
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.tight_layout()
    plt.show()
    #fig.savefig(save_fig_2)
    
    if meas_type == 3:
        
        covar_env_upper3 = abs(prefit_bounds_list[:stop_index, 2])*3
        
        fig_preFit_range = plt.figure()
        plt.plot(times, covar_env_upper3, label='_nolegend_', c='g')
        plt.plot(times, -covar_env_upper3, label='_nolegend_', c='g')
        plt.scatter(times[indices_1], pre_fit_list_new[indices_1, 2], s=50, c='m', marker='s')
        plt.scatter(times[indices_2], pre_fit_list_new[indices_2, 2], s=50, c='g', marker='^')
        plt.scatter(times[indices_3], pre_fit_list_new[indices_3, 2], s=50, c='r', marker='D')
        plt.scatter(times[indices_4], pre_fit_list_new[indices_4, 2], s=50, c='k', marker='o')
        plt.ylabel('Range Residuals (km)', fontsize=18)
        plt.xlabel(time_str, fontsize=18)
        plt.title('Range pre-fit Residuals', fontsize=18)
        plt.legend(legend_names, fontsize=10)
        #plt.ylim([-.01,.01])
        plt.xlim([times[0] - 5, times[-1] + 5])
        plt.tight_layout()
        plt.show()
        #fig.savefig('prefit_range.png')
    
    if saveFig_bool:
        fig_preFit_az.savefig('Figures/preFit_az.png')
        fig_preFit_el.savefig('Figures/preFit_el.png')
        fig_preFit_range.savefig('Figures/preFit_range.png')
        
        
def calc_display_results(post_fit_list, measurement_array, R, meas_type, stop_index, saveFig_bool, time_str):
    
    
    rms_1 = 'Range ='
    unit_1 = 'km'
    ylabel_1 = 'Range Residuals (km)'
    title_1 = 'Range Post-fit Residuals'
    save_fig_1 = 'postfit_range.png'
    rms_2 = 'Range Rate ='
    unit_2 = 'km/s'
    ylabel_2 = 'Range Rate Residuals (km/s)'
    title_2 = 'Range Rate Post-fit Residuals'
    save_fig_2 = 'postfit_rangeRate.png'
    post_fit_list_new = copy.deepcopy(post_fit_list)
         
    if (meas_type == 2) or (meas_type == 3):
        rms_1 = 'Azimuth ='
        unit_1 = 'degrees'
        rms_2 = 'Elevation ='
        unit_2 = 'degrees'
        post_fit_list_new[:, 0] = np.degrees(post_fit_list[:, 0])
        post_fit_list_new[:, 1] = np.degrees(post_fit_list[:, 1])
        ylabel_1 = 'Azimuth Residuals (degrees)'
        title_1 = 'Azimuth Post-fit Residuals'
        save_fig_1 = 'postfit_az.png'
        ylabel_2 = 'Elevation Residuals (degrees)'
        title_2 = 'Elevation Post-fit Residuals'
        save_fig_2 = 'postfit_el_rate.png'

        
    

    times = measurement_array[:stop_index,0]/(60)
    
    indices_1 = np.where(measurement_array[:stop_index, 1] == 1)[0]
    indices_2 = np.where(measurement_array[:stop_index, 1] == 2)[0]
    indices_3 = np.where(measurement_array[:stop_index, 1] == 3)[0]
    indices_4 = np.where(measurement_array[:stop_index, 1] == 4)[0]

    
    
    #Post-fit
    print('Post-fit RMS:')
    post_fit_1_list_4RMS = post_fit_list_new[:, 0]
    postfit_1_rms = np.sqrt(np.mean(np.square(post_fit_1_list_4RMS)))
    print(rms_1, "%.4f" % postfit_1_rms, unit_1)

    post_fit_2_list_4RMS = post_fit_list_new[:, 1]
    postfit_2_rms = np.sqrt(np.mean(np.square(post_fit_2_list_4RMS)))
    print(rms_2, "%.4f" % postfit_2_rms, unit_2)
    
    if meas_type == 3:
        post_fit_3_list_4RMS = post_fit_list_new[:, 2]
        postfit_3_rms = np.sqrt(np.mean(np.square(post_fit_3_list_4RMS)))
        print('Range =', "%.5f" % postfit_3_rms, 'km')
    
    
    covar_env_upper1 = np.ones((stop_index)) * np.degrees(np.sqrt(abs(R[0, 0])))*3
    covar_env_upper2 = np.ones((stop_index)) * np.degrees(np.sqrt(abs(R[1, 1])))*3

   
    #Post-fit Residuals
    fig_postFit_az = plt.figure()
    plt.plot(times, covar_env_upper1, label='_nolegend_', c='g')
    plt.plot(times, -covar_env_upper1, label='_nolegend_', c='g')
    plt.scatter(times[indices_1], post_fit_list_new[indices_1, 0], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], post_fit_list_new[indices_2, 0], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], post_fit_list_new[indices_3, 0], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], post_fit_list_new[indices_4, 0], s=50, c='k', marker='o')
    plt.ylabel(ylabel_1, fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.title(title_1, fontsize=18)
    legend_names = ['Station 1', 'Station 2']
    plt.legend(legend_names, fontsize=10)
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.tight_layout()
    plt.show()
    #fig.savefig(save_fig_1)

    fig_postFit_el = plt.figure()
    plt.plot(times, covar_env_upper2, label='_nolegend_', c='g')
    plt.plot(times, -covar_env_upper2, label='_nolegend_', c='g')
    plt.scatter(times[indices_1], post_fit_list_new[indices_1, 1], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], post_fit_list_new[indices_2, 1], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], post_fit_list_new[indices_3, 1], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], post_fit_list_new[indices_4, 1], s=50, c='k', marker='o')
    plt.ylabel(ylabel_2, fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.title(title_2, fontsize=18)
    plt.legend(legend_names, fontsize=10)
    #plt.ylim([-.01,.01])
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.tight_layout()
    plt.show()
    #fig.savefig(save_fig_2)
    
    if meas_type == 3:
        
        covar_env_upper3 = np.ones((stop_index)) * np.sqrt(abs(R[2, 2]))*3
        
        fig_postFit_range = plt.figure()
        plt.plot(times, covar_env_upper3, label='_nolegend_', c='g')
        plt.plot(times, -covar_env_upper3, label='_nolegend_', c='g')
        plt.scatter(times[indices_1], post_fit_list_new[indices_1, 2], s=50, c='m', marker='s')
        plt.scatter(times[indices_2], post_fit_list_new[indices_2, 2], s=50, c='g', marker='^')
        plt.scatter(times[indices_3], post_fit_list_new[indices_3, 2], s=50, c='r', marker='D')
        plt.scatter(times[indices_4], post_fit_list_new[indices_4, 2], s=50, c='k', marker='o')
        plt.ylabel('Range Residuals (km)', fontsize=18)
        plt.xlabel(time_str, fontsize=18)
        plt.title('Range Post-fit Residuals', fontsize=18)
        legend_names = ['Station 1', 'Station 2']
        plt.legend(legend_names, fontsize=10)
        #plt.ylim([-.01,.01])
        plt.xlim([times[0] - 5, times[-1] + 5])
        plt.tight_layout()
        plt.show()
        #fig.savefig('postfit_range.png')
    
    if saveFig_bool:
        fig_postFit_az.savefig('Figures/postFit_az.png')
        fig_postFit_el.savefig('Figures/postFit_el.png')
        fig_postFit_range.savefig('Figures/postFit_range.png')
        
    
    
def plot_error_covar_xref(P_list, x_ref_updated_list, obs_data_truth, density_truth, x_range, y_range, \
                          z_range, xv_range, yv_range, zv_range, measurement_array, time, stop_index, saveFig_bool, time_str):
    
    #Compare to the Truth Data : Estimation Errors------

    times = time[:stop_index]/(60)
    
    indices_1 = np.where(measurement_array[:stop_index, 1] == 1)[0]
    indices_2 = np.where(measurement_array[:stop_index, 1] == 2)[0]
    indices_3 = np.where(measurement_array[:stop_index, 1] == 3)[0]
    indices_4 = np.where(measurement_array[:stop_index, 1] == 4)[0]
    
    density_covar_env_upper = np.sqrt(abs(P_list[:stop_index, -1, -1]))*3
    density_covar_env_lower = -density_covar_env_upper
    density_error = x_ref_updated_list[:stop_index,-1] - density_truth[:stop_index]
    
    x_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 0, 0]))*3
    x_covar_env_lower = -x_covar_env_upper
    x_error = x_ref_updated_list[:stop_index,0] - obs_data_truth[:stop_index, 0]
    
    y_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 1, 1]))*3
    y_covar_env_lower = -y_covar_env_upper
    y_error = x_ref_updated_list[:stop_index,1] - obs_data_truth[:stop_index, 1]
    
    z_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 2, 2]))*3
    z_covar_env_lower = -z_covar_env_upper
    z_error = x_ref_updated_list[:stop_index,2] - obs_data_truth[:stop_index, 2]
    
    #error_pos_norm = np.sqrt(x_error**2 + y_error**2 + z_error**2)
    error_density_rms_3D = np.sqrt(np.mean(np.square(density_error)))
    print('Density RMS =', "%.5f" % error_density_rms_3D, r'$(kg/km^3)$')
    
    print('Position RMS:')
    error_x_pos_rms_3D = np.sqrt(np.mean(np.square(x_error)))
    print('X =', "%.4f" % error_x_pos_rms_3D, 'km')
    
    error_y_pos_rms_3D = np.sqrt(np.mean(np.square(y_error)))
    print('Y =', "%.4f" % error_y_pos_rms_3D, 'km')
    
    error_z_pos_rms_3D = np.sqrt(np.mean(np.square(z_error)))
    print('Z =', "%.4f" % error_z_pos_rms_3D, 'km')
    
    pos_rms = np.sqrt(error_x_pos_rms_3D**2 + error_y_pos_rms_3D**2 + error_z_pos_rms_3D**2)
    print('Overall =', "%.4f" % pos_rms, 'km')
    
    
    #Density
    fig_dens = plt.figure()
    plt.plot(times, density_covar_env_upper, label='_nolegend_', c='g')
    plt.plot(times, density_covar_env_lower, label='_nolegend_', c='g')
    plt.scatter(times[indices_1], density_error[indices_1], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], density_error[indices_2], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], density_error[indices_3], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], density_error[indices_4], s=50, c='k', marker='o')
    plt.ylabel(r'$(kg/km^3)$', fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    legend_names = ['Station 1', 'Station 2']
    plt.legend(legend_names, fontsize=10)
    plt.title('Reduced Rank EnKF Density Error & Covariance Envelope', fontsize=18)
    #plt.ylim([-x_range,x_range])
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.show()
    

    #x Position
    fig_xpos = plt.figure()
    plt.plot(times, x_covar_env_upper, label='_nolegend_', c='g')
    plt.plot(times, x_covar_env_lower, label='_nolegend_', c='g')
    
    plt.scatter(times[indices_1], x_error[indices_1], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], x_error[indices_2], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], x_error[indices_3], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], x_error[indices_4], s=50, c='k', marker='o')
    plt.ylabel('km', fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.legend(legend_names, fontsize=10)
    plt.title('Reduced Rank EnKF X Position Error & Covariance Envelope', fontsize=18)
    plt.ylim([-x_range,x_range])
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.show()

    #y Position 
    fig_ypos = plt.figure()
    plt.plot(times, y_covar_env_upper, label='_nolegend_', c='g')
    plt.plot(times, y_covar_env_lower, label='_nolegend_', c='g')
    plt.scatter(times[indices_1], y_error[indices_1], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], y_error[indices_2], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], y_error[indices_3], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], y_error[indices_4], s=50, c='k', marker='o')
    plt.ylabel('km', fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.legend(legend_names, fontsize=10)
    plt.title('EnKF Y Position Error & Covariance Envelope', fontsize=18)
    plt.ylim([-y_range,y_range])
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.show()
    #fig.savefig('y_pos_error.png')

    #z Position
    fig_zpos = plt.figure()
    plt.plot(times, z_covar_env_upper, label='_nolegend_', c='g')
    plt.plot(times, z_covar_env_lower, label='_nolegend_', c='g')
    plt.scatter(times[indices_1], z_error[indices_1], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], z_error[indices_2], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], z_error[indices_3], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], z_error[indices_4], s=50, c='k', marker='o')
    plt.ylabel('km', fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.legend(legend_names, fontsize=10)
    plt.title('EnKF Z Position Error & Covariance Envelope', fontsize=18)
    plt.ylim([-z_range,z_range])
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.show()
    
    #x Velocity
    x_dot_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 3, 3]))*3
    x_dot_covar_env_lower = -np.sqrt(abs(P_list[:stop_index, 3, 3]))*3
    x_vel_error = x_ref_updated_list[:stop_index,3] - obs_data_truth[:stop_index, 3]

    fig_xvel = plt.figure()
    plt.plot(times, x_dot_covar_env_upper, label='_nolegend_', c='g')
    plt.plot(times, x_dot_covar_env_lower, label='_nolegend_', c='g')
    plt.scatter(times[indices_1], x_vel_error[indices_1], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], x_vel_error[indices_2], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], x_vel_error[indices_3], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], x_vel_error[indices_4], s=50, c='k', marker='o')
    plt.ylabel('km/second', fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.legend(legend_names, fontsize=10)
    plt.title('EnKF X Velocity Error & Covariance Envelope', fontsize=18)
    plt.ylim([-xv_range,xv_range])
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.show()
    #fig.savefig('x_vel_error.png')

    #y Velocity
    y_dot_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 4, 4]))*3
    y_dot_covar_env_lower = -np.sqrt(abs(P_list[:stop_index, 4, 4]))*3
    y_vel_error = x_ref_updated_list[:stop_index,4] - obs_data_truth[:stop_index, 4]

    fig_yvel = plt.figure()
    plt.plot(times, y_dot_covar_env_upper, label='_nolegend_', c='g')
    plt.plot(times, y_dot_covar_env_lower, label='_nolegend_', c='g')
    plt.scatter(times[indices_1], y_vel_error[indices_1], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], y_vel_error[indices_2], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], y_vel_error[indices_3], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], y_vel_error[indices_4], s=50, c='k', marker='o')
    plt.ylabel('km/second', fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.legend(legend_names, fontsize=10)
    plt.title('EnKF Y Velocity Error & Covariance Envelope', fontsize=18)
    plt.ylim([-yv_range,yv_range])
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.show()

    #z Velocity
    z_dot_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 5, 5]))*3
    z_dot_covar_env_lower = -np.sqrt(abs(P_list[:stop_index, 5, 5]))*3
    z_vel_error = x_ref_updated_list[:stop_index,5] - obs_data_truth[:stop_index, 5]

    fig_zvel = plt.figure()
    plt.plot(times, z_dot_covar_env_upper, label='_nolegend_', c='g')
    plt.plot(times, z_dot_covar_env_lower, label='_nolegend_', c='g') 
    plt.scatter(times[indices_1], z_vel_error[indices_1], s=50, c='m', marker='s')
    plt.scatter(times[indices_2], z_vel_error[indices_2], s=50, c='g', marker='^')
    plt.scatter(times[indices_3], z_vel_error[indices_3], s=50, c='r', marker='D')
    plt.scatter(times[indices_4], z_vel_error[indices_4], s=50, c='k', marker='o')
    plt.ylabel('km/second', fontsize=18)
    plt.xlabel(time_str, fontsize=18)
    plt.legend(legend_names, fontsize=10)
    plt.title('EnKF Z Velocity Error & Covariance Envelope', fontsize=18)
    plt.ylim([-zv_range,zv_range])
    plt.xlim([times[0] - 5, times[-1] + 5])
    plt.show()
    
    
    print('Velocity RMS:')
    error_x_vel_rms_3D = np.sqrt(np.mean(np.square(x_vel_error)))
    print('X =', "%.6f" % error_x_vel_rms_3D, 'km/second')
    
    error_y_vel_rms_3D = np.sqrt(np.mean(np.square(y_vel_error)))
    print('Y =', "%.6f" % error_y_vel_rms_3D, 'km/second')
    
    error_z_vel_rms_3D = np.sqrt(np.mean(np.square(z_vel_error)))
    print('Z =', "%.6f" % error_z_vel_rms_3D, 'km/second')
    
    vel_rms = np.sqrt(error_x_vel_rms_3D**2 + error_y_vel_rms_3D**2 + error_z_vel_rms_3D**2)
    print('Overall =', "%.6f" % vel_rms, 'km/s')

    
    if saveFig_bool:
        fig_dens.savefig('Figures/dens_error.png')
        fig_xpos.savefig('Figures/x_pos_error.png')
        fig_ypos.savefig('Figures/y_pos_error.png')
        fig_zpos.savefig('Figures/z_pos_error.png')
        fig_xvel.savefig('Figures/x_vel_error.png')
        fig_yvel.savefig('Figures/y_vel_error.png')
        fig_zvel.savefig('Figures/z_vel_error.png')
    
        

In [ ]:

#Generate Plots for Analysis

x_range = .05
y_range = .05
z_range = .05

xv_range = 1e-4
yv_range = 1e-4
zv_range = 1e-4

times = measurement_array[:, 0]


calc_display_results_pre(pre_fit_list, prefit_bounds_list, measurement_array, R, meas_type, stop_index, saveFig_bool, time_str)


calc_display_results(post_fit_list_EnKF, measurement_array, R, meas_type, stop_index, saveFig_bool,\
                                     time_str)


plot_error_covar_xref(P_list_EnKF, X_mean_updated_list_EnKF, \
                      truth_xyz, true_density_array, x_range, y_range, z_range, xv_range, yv_range, zv_range,\
                      measurement_array, times, stop_index, saveFig_bool, time_str)


In [ ]:
indices = np.where(measurement_array[:, 1] == 1)[0]
print(indices)
indices = np.where(measurement_array[:, 1] == 2)[0]
print(indices)
indices = np.where(measurement_array[:, 1] == 3)[0]
print(indices)
indices = np.where(measurement_array[:, 1] == 4)[0]
print(indices)